



# Applied Data Science : Capstone Project - Snacks On Wheels





### Description :
    
This project provide data analysis support for “Snacks on wheels” , a mobile snack shop, to decide the sales spot/location within the  neighborhood of city of New York based on following criteria.  

  
    a.Top 10 Most Trending Venue ( Expect restaurants )  
    b.Nearest ( min distance)  from current sales spots  
    c.Venue Rating  
    d.Number of tips /  tips details  
    e.Consulting/contacting user who gave most valuable tip ( if needed)  
    f.Other popular spots near the venue  

#####  Let us help them to find the best sales spot !!!


#### Step  1: Import required Libraries

In [19]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### Step  2 : Load location data

In [20]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [21]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [22]:
neighborhoods_data = newyork_data['features']

In [23]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [24]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

#### Step 3: Scrap the data , clean it and preprocess

In [25]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [26]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Step 4: Find the location of New York City , and plot Neighborhood

In [27]:
address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of New York City are 40.7308619, -73.9871558.


Draw a map of Toronto using latitude and longitude values

In [28]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Limit the scope of this project to Borough contain "Manhattan".

In [29]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()


,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [30]:
manhattan_data.shape

(40, 4)

####  Step 5 : Random selection of current location for test.

In [31]:
cur_loc=manhattan_data ['Neighborhood'].sample()
latitude=manhattan_data['Latitude'].loc[manhattan_data['Neighborhood'].isin(cur_loc)] .values[0]
longitude=manhattan_data['Longitude'].loc[manhattan_data['Neighborhood'].isin(cur_loc)].values[0]
cur_Borough=manhattan_data['Borough'].loc[manhattan_data['Neighborhood'].isin(cur_loc)].values[0]
print(latitude, longitude)
print (cur_Borough)


40.721521967443216 -74.01068328559087
Manhattan


####  Step 6: Defining Foursqure credentials 

In [32]:
CLIENT_ID = 'YM4DEIRN1QRSQKLOKOMVW5LOFGR32DJPPSP2ASJOSPG5ISV5' # your Foursquare ID
CLIENT_SECRET = 'XWG2KZR0RV0PNUSC3JUWLQA3RFWPJBLRGTKC2V1BHNKDYSYD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YM4DEIRN1QRSQKLOKOMVW5LOFGR32DJPPSP2ASJOSPG5ISV5
CLIENT_SECRET:XWG2KZR0RV0PNUSC3JUWLQA3RFWPJBLRGTKC2V1BHNKDYSYD


#### Step 7: Accessing  Foursqure API for Explore Trending Venues

In [33]:
# Seraching within the radius of 2000 meters
radius = 2000
# limit the number of records to 50
LIMIT=100

#### Step 8: Sending request for trending location details & format result

In [34]:
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION,  radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/trending?client_id=YM4DEIRN1QRSQKLOKOMVW5LOFGR32DJPPSP2ASJOSPG5ISV5&client_secret=XWG2KZR0RV0PNUSC3JUWLQA3RFWPJBLRGTKC2V1BHNKDYSYD&ll=40.721521967443216,-74.01068328559087&v=20180605&radius=2000&limit=100'

In [35]:
results = requests.get(url).json()
results
x=len(results['response']['venues'])
x
if len(results['response']['venues']) == 0:
    dataframe= 'No trending venues are available at the moment!'
  
else:
     # assign relevant part of JSON to venues
        venues = results['response']['venues']

        # tranform venues into a dataframe
        dataframe = json_normalize(venues)
        dataframe.head()

dataframe 

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name
0,"[{'id': '4bf58dd8d48988d1bd941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,5484890c498e985cf4e3c076,100 Kenmare St,US,New York,United States,btwn Cleveland & Mulberry St,1145,[100 Kenmare St (btwn Cleveland & Mulberry St)...,"[{'label': 'display', 'lat': 40.7211844, 'lng'...",40.721184,-73.997111,NaN,10012,NY,sweetgreen
1,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",NaN,NaN,NaN,NaN,NaN,NaN,49bc3b0af964a52020541fe3,270 Greenwich Street,US,New York,United States,at Warren St,647,"[270 Greenwich Street (at Warren St), New York...",NaN,40.715877,-74.012514,Tribeca,10007,NY,Whole Foods Market
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",292476,/delivery_provider_seamless_20180129.png,https://igx.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/aroma-espresso-b...,4edd43af7bebc29455367992,100 Church St,US,New York,United States,at Barclay St,913,"[100 Church St (at Barclay St), New York, NY 1...","[{'label': 'display', 'lat': 40.7133292, 'lng'...",40.713329,-74.010158,NaN,10007,NY,Aroma Espresso Bar


In [36]:
if len(results['response']['venues']) == 0:
    dataframe_filtered= 'No trending venues are available at the moment!'
else:
    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]
    
    # function that extracts the category of the venue
    def get_category_type(row):
        try:
            categories_list = row['categories']
        except:
            categories_list = row['venue.categories']
            
        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']
    
    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,sweetgreen,Salad Place,100 Kenmare St,US,New York,United States,btwn Cleveland & Mulberry St,1145,[100 Kenmare St (btwn Cleveland & Mulberry St)...,"[{'label': 'display', 'lat': 40.7211844, 'lng'...",40.721184,-73.997111,NaN,10012,NY,5484890c498e985cf4e3c076
1,Whole Foods Market,Grocery Store,270 Greenwich Street,US,New York,United States,at Warren St,647,"[270 Greenwich Street (at Warren St), New York...",NaN,40.715877,-74.012514,Tribeca,10007,NY,49bc3b0af964a52020541fe3
2,Aroma Espresso Bar,Coffee Shop,100 Church St,US,New York,United States,at Barclay St,913,"[100 Church St (at Barclay St), New York, NY 1...","[{'label': 'display', 'lat': 40.7133292, 'lng'...",40.713329,-74.010158,NaN,10007,NY,4edd43af7bebc29455367992


#### Step 8 : Remove the data if  Category of Trending location contain "Restaurant"

In [37]:
if len(results['response']['venues']) == 0:
    dataframe_filtered= 'No trending venues are available at the moment!'
    
else:
    dataframe_filtered=dataframe_filtered[dataframe_filtered.categories != 'Restaurant']
    dataframe_filtered.head()
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,sweetgreen,Salad Place,100 Kenmare St,US,New York,United States,btwn Cleveland & Mulberry St,1145,[100 Kenmare St (btwn Cleveland & Mulberry St)...,"[{'label': 'display', 'lat': 40.7211844, 'lng'...",40.721184,-73.997111,NaN,10012,NY,5484890c498e985cf4e3c076
1,Whole Foods Market,Grocery Store,270 Greenwich Street,US,New York,United States,at Warren St,647,"[270 Greenwich Street (at Warren St), New York...",NaN,40.715877,-74.012514,Tribeca,10007,NY,49bc3b0af964a52020541fe3
2,Aroma Espresso Bar,Coffee Shop,100 Church St,US,New York,United States,at Barclay St,913,"[100 Church St (at Barclay St), New York, NY 1...","[{'label': 'display', 'lat': 40.7133292, 'lng'...",40.713329,-74.010158,NaN,10007,NY,4edd43af7bebc29455367992


#### Step 9: Presenting the data analysis result to "Snacks on Wheel" Business Owner

In [38]:
if len(results['response']['venues']) == 0:
    Trending_loc= 'No trending venues are available at the moment!'
else:
    Trending_loc = dataframe_filtered[['name', 'categories', 'address','city','distance','lat','lng']].copy()

print ('*************************************TRENDING VENUS ************************************************************')    
Trending_loc




*************************************TRENDING VENUS ************************************************************


,name,categories,address,city,distance,lat,lng
0,sweetgreen,Salad Place,100 Kenmare St,New York,1145,40.721184,-73.997111
1,Whole Foods Market,Grocery Store,270 Greenwich Street,New York,647,40.715877,-74.012514
2,Aroma Espresso Bar,Coffee Shop,100 Church St,New York,913,40.713329,-74.010158


In [39]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'
    
else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the trending location

    # add a red circle marker to represent the current location
    folium.features.CircleMarker(
    [latitude, longitude],
    radius=20,
    color='red',
    popup='Current Location',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
    ).add_to(venues_map)
    
    # add the trending location as blue circle markers
    for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
        folium.features.CircleMarker(
        [lat, lng],
        radius=7,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

###  Nearest Trending Venue Selection Based on Stat. Min distance

In [40]:
ShortPath=dataframe_filtered['distance'].idxmin()

print ('$$$$$$$$$$$$$$$$$$$$$$$$$$$$ Nearest Trending Venue $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')

dataframe_filtered['name'].iloc[ShortPath]

$$$$$$$$$$$$$$$$$$$$$$$$$$$$ Nearest Trending Venue $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$


'Whole Foods Market'

#### Step 10 : Analyze the Trending Venue Further by Explore near by area.

In [41]:
venue_id = dataframe_filtered['id'].iloc[ShortPath]

url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/49bc3b0af964a52020541fe3?client_id=YM4DEIRN1QRSQKLOKOMVW5LOFGR32DJPPSP2ASJOSPG5ISV5&client_secret=XWG2KZR0RV0PNUSC3JUWLQA3RFWPJBLRGTKC2V1BHNKDYSYD&v=20180605'

In [42]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'storeId', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '49bc3b0af964a52020541fe3',
 'name': 'Whole Foods Market',
 'contact': {'phone': '2123496555',
  'formattedPhone': '(212) 349-6555',
  'twitter': 'wholefoodsnyc',
  'facebook': '107822261014',
  'facebookUsername': 'wholefoodsnyc',
  'facebookName': 'Whole Foods Market NYC'},
 'location': {'address': '270 Greenwich Street',
  'crossStreet': 'at Warren St',
  'lat': 40.71587659935159,
  'lng': -74.01251422178355,
  'postalCode': '10007',
  'cc': 'US',
  'neighborhood': 'Tribeca',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['270 Greenwich Street (at Warren St)',
   'New York, NY 10007',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/whole-foods-market/49bc3b0af964a52020541fe3',
 'categories': [{'id': '4bf58dd8d48988d118951735',
   'name': 'Grocery Store',
   'pluralName': 'Grocery Stores',
   'shortName': 'Grocery Store',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
    'suffix

### Venue Rating 

In [43]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.3


###  Number of tips

In [44]:
result['response']['venue']['tips']['count']

247

### Venue's tips in details

In [45]:
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bb26a10351e3d4c8e6037fb'},
 'response': {'tips': {'count': 247,
   'items': [{'id': '51aa7374498ef112a06b5839',
     'createdAt': 1370125172,
     'text': "If you've forgotten what a normal grocery experience is like, come here and spread out in this spacious and well-organized location; best in NY. Full of all the delicious items your heart could desire",
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/51aa7374498ef112a06b5839',
     'lang': 'en',
     'likes': {'count': 4,
      'groups': [{'type': 'others',
        'count': 4,
        'items': [{'id': '94146557',
          'firstName': 'Edwin',
          'lastName': 'Toure',
          'gender': 'male',
          'photo': {'prefix': 'https://igx.4sqi.net/img/user/',
           'suffix': '/94146557-NNVQFXMZDTQSC5N1'}},
         {'id': '3772339',
          'firstName': 'Olivier',
          'lastName': 'Kitenge',
          'gender': 'none',
          'photo': {'prefix': 'https

In [46]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'lastVoteText', 'lastUpvoteTimestamp', 'todo', 'user'])

In [47]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]


# display tips
print("############################ Venue's Tips###########################")

tips_filtered

############################ Venue's Tips###########################


/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"If you've forgotten what a normal grocery experience is like, come here and spread out in this spacious and well-organized location; best in NY. Full of all the delicious items your heart could desire",9,0,51aa7374498ef112a06b5839,Mary Elise Chavez,NaN,female,620779


### Analyze about the user who gave the tip 

In [48]:
user_id = tips_filtered['user.id'].iloc[0] # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()


dict_keys(['id', 'firstName', 'gender', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'photos', 'type', 'mayorships', 'checkins', 'lists', 'lenses'])

### User Name & Contact details for further details if needed

In [49]:
print('First Name: ' + user_data['firstName'])
print('Home City: ' + user_data['homeCity'])
print('contact: ' + str(user_data['contact']))

First Name: Mary Elise Chavez
Home City: New York, NY
contact: {}


#### Step 11 :  Explore the location Further before final decision



In [50]:
latitude = dataframe_filtered['lat'].iloc[ShortPath]
longitude = dataframe_filtered['lng'].iloc[ShortPath]
LIMIT=20
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url



'https://api.foursquare.com/v2/venues/explore?client_id=YM4DEIRN1QRSQKLOKOMVW5LOFGR32DJPPSP2ASJOSPG5ISV5&client_secret=XWG2KZR0RV0PNUSC3JUWLQA3RFWPJBLRGTKC2V1BHNKDYSYD&ll=40.71587659935159,-74.01251422178355&v=20180605&radius=2000&limit=20'

In [51]:
import requests

In [52]:
results = requests.get(url).json()
'There are {} landmarks around this venue.'.format(len(results['response']['groups'][0]['items']))

'There are 20 landmarks around this venue.'

In [53]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4a9bcc2ff964a5203b3520e3',
  'name': 'Washington Market Park',
  'location': {'address': 'Greenwich St',
   'crossStreet': 'at Reade St',
   'lat': 40.71704598853704,
   'lng': -74.01109457015991,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.71704598853704,
     'lng': -74.01109457015991}],
   'distance': 176,
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['Greenwich St (at Reade St)',
    'New York, NY',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1e7941735',
    'name': 'Playground',
    'pluralName': 'Playgrounds',
    'shortName': 'Playground',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/playground_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'refer

###   Popular Spots  Near BY Trending Venue

In [54]:
dataframe_E = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe_E.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered_E = dataframe_E.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered_E['venue.categories'] = dataframe_filtered_E.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered_E.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

print ('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ Popular Spots  Near BY Trending Venue Area @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')

dataframe_filtered_E.head(10)

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ Popular Spots  Near BY Trending Venue Area @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Washington Market Park,Playground,Greenwich St,US,New York,United States,at Reade St,176,"[Greenwich St (at Reade St), New York, NY, United States]","[{'label': 'display', 'lat': 40.71704598853704, 'lng': -74.01109457015991}]",40.717046,-74.011095,NaN,NaN,NY,4a9bcc2ff964a5203b3520e3
1,Whole Foods Market,Grocery Store,270 Greenwich Street,US,New York,United States,at Warren St,0,"[270 Greenwich Street (at Warren St), New York, NY 10007, United States]",NaN,40.715877,-74.012514,Tribeca,10007,NY,49bc3b0af964a52020541fe3
2,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,US,New York,United States,btwn Greenwich St & W Broadway,168,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}]",40.714788,-74.011132,NaN,10007,NY,54148bc6498ea7bb8c05b70a
3,Mulberry & Vine,Café,73 Warren St,US,New York,United States,btwn W Broadway & Greenwich St,208,"[73 Warren St (btwn W Broadway & Greenwich St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71517693966315, 'lng': -74.01022747778285}]",40.715177,-74.010227,NaN,10007,NY,5171b5cc011cef9833bbb787
4,Tribeca Performing Arts Center,Performing Arts Venue,199 Chambers St,US,New York,United States,at West Side Hwy,193,"[199 Chambers St (at West Side Hwy), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.7175939490532, 'lng': -74.01219801457627}]",40.717594,-74.012198,NaN,10007,NY,49f7456af964a5207b6c1fe3
5,Asphalt Green Battery Park City,Athletics & Sports,212 N End Ave,US,New York,United States,Murray Street,182,"[212 N End Ave (Murray Street), New York, NY 10282, United States]","[{'label': 'display', 'lat': 40.715923644223366, 'lng': -74.01467345576101}]",40.715924,-74.014673,NaN,10282,NY,51bcd723498eecb51dd996bc
6,Chambers Street Wines,Wine Shop,148 Chambers St,US,New York,United States,btwn West Broadway & Hudson St,236,"[148 Chambers St (btwn West Broadway & Hudson St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.715773063928374, 'lng': -74.00971823312332}]",40.715773,-74.009718,NaN,10007,NY,4adcf23cf964a520cc6221e3
7,SoulCycle TriBeCa,Cycle Studio,103 Warren St,US,New York,United States,at West Side Hwy,37,"[103 Warren St (at West Side Hwy), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71604284303925, 'lng': -74.01290442320492}]",40.716043,-74.012904,NaN,10007,NY,4b520135f964a520aa6027e3
8,Shake Shack,Burger Joint,215 Murray St,US,New York,United States,btwn West St & North End Ave,207,"[215 Murray St (btwn West St & North End Ave), New York, NY 10282, United States]",NaN,40.715257,-74.014828,NaN,10282,NY,4dda977ad22d38ef4332c419
9,Kaffe 1668,Coffee Shop,275 Greenwich St,US,New York,United States,at Murray St,119,"[275 Greenwich St (at Murray St), New York, NY 10007, United States]",NaN,40.715202,-74.011412,NaN,10007,NY,49ccd495f964a52091591fe3


###  Map : Popular Spots  Near BY Trending Venue

In [55]:
 venues_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around the trending location

# add a red circle marker to represent the current location
folium.features.CircleMarker(
    [latitude, longitude],
    radius=20,
    color='red',
    popup='Trending Venue-shortest distance',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
    ).add_to(venues_map)
    
# add the trending location as blue circle markers
for lat, lng, label in zip(dataframe_filtered_E.lat, dataframe_filtered_E.lng, dataframe_filtered_E.categories):
        folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)


# display map
venues_map

## Now choose wisely, do you want to move to nearest trending location or continue here? We advise a move based on this analysis

### Thanks You - End of this Project
